# Hypertuning of parameters for DeepAR Estimator

### Imports and housekeeping

In [51]:
import sagemaker

from sagemaker.estimator import Estimator
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.amazon.amazon_estimator import get_image_uri

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

### Helper functino to return a properly initiated DeepAR estimator

In [86]:
def deepar_estimator(prediction_length):
    image_name = get_image_uri(sagemaker_session.boto_region_name, 'forecasting-deepar')    
    estimator = Estimator(sagemaker_session=sagemaker_session,
                            image_name=image_name,
                            role=role,
                            train_instance_count=1,
                            train_instance_type='ml.c4.xlarge',
                            train_use_spot_instances=True,
                            train_max_wait=3600,
                            train_max_run=3600)
    
    hyperparameters_base = {
        "epochs": "100",
        "time_freq": 'W',
        "num_layers": "2",
        "early_stopping_patience": "10",
        'prediction_length': prediction_length,
        'num_dynamic_feat': 'auto',
        'cardinality': 'ignore'}
    
    estimator.set_hyperparameters(**hyperparameters_base)
    
    return estimator

### Hyperparameters search grid, objective metric and input definitions

In [81]:
objective_metric_name = 'test:RMSE'

hyperparameter_ranges_sj = {
    'dropout_rate': ContinuousParameter(0.03, 0.3),
    'learning_rate': ContinuousParameter(0.001, 0.3),
    'mini_batch_size': IntegerParameter(32, 128),
    'context_length': IntegerParameter(52, 260),
    'num_cells': IntegerParameter(30, 128)
}

hyperparameter_ranges_iq = {
    'dropout_rate': ContinuousParameter(0.03, 0.3),
    'learning_rate': ContinuousParameter(0.001, 0.3),
    'mini_batch_size': IntegerParameter(32, 128),
    'context_length': IntegerParameter(26, 156),
    'num_cells': IntegerParameter(30, 128)
}

In [82]:
data_channels_sj = {'train': 's3://sagemaker-eu-central-1-964501460451/dengai/2020-04-15--01-10-dew-std-label/pprocess_data/train_pp_sj.json',
                 'test' : 's3://sagemaker-eu-central-1-964501460451/dengai/2020-04-15--01-10-dew-std-label/pprocess_data/train_pp_sj.json' }
    
data_channels_iq = {'train': 's3://sagemaker-eu-central-1-964501460451/dengai/2020-04-15--01-10-dew-std-label/pprocess_data/train_test_pp_iq.json',
                 'test' : 's3://sagemaker-eu-central-1-964501460451/dengai/2020-04-15--01-10-dew-std-label/pprocess_data/train_test_pp_iq.json' }

### Estimator and hypertuning job instanciation

In [87]:
estimator_sj = deepar_estimator(260)
estimator_iq = deepar_estimator(156)

In [90]:
tuner_sj = HyperparameterTuner(estimator_sj,
                            objective_metric_name,
                            hyperparameter_ranges_sj,
                            max_jobs=50,
                            max_parallel_jobs=10,
                            objective_type='Minimize',
                            base_tuning_job_name='sj'
                            )
tuner_iq = HyperparameterTuner(estimator_iq,
                            objective_metric_name,
                            hyperparameter_ranges_iq,
                            max_jobs=50,
                            max_parallel_jobs=10,
                            objective_type='Minimize',
                            base_tuning_job_name='iq')

### Start and wait for the hypertuning jobs

In [ ]:
tuner_sj.fit(data_channels_sj)
tuner_iq.fit(data_channels_iq)

In [92]:
tuner_iq.wait()
tuner_sj.wait()

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

## Getting hypertuning job information and visualization

In [9]:
def analyze_ht_job(job_name):
    ht = HyperparameterTuner.attach(job_name)
    stats = ht.analytics().dataframe()
    print('Number of training jobs and parameters searched \n')
    print(stats.nunique())
    print('\n \n')
    
    stats_for_plot = stats[(stats.TrainingJobStatus=='Completed')].sort_values('FinalObjectiveValue')[:-5]
    best_ix = stats_for_plot.FinalObjectiveValue.idxmin()
    stats_for_plot.loc[best_ix]
    
    markers = ['o' for _ in range(len(stats_for_plot))]
    markers[0] = 'X'
    sns.pairplot(data=stats_for_plot, vars=ht._hyperparameter_ranges.keys(), hue='FinalObjectiveValue', palette='RdBu_r', markers=markers)

In [ ]:
analyze_ht_job(tuner_iq._current_job_name)